In [116]:
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout

In [117]:
# Image Widget

file = open("../logo.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='200'
                )

label_headline = widgets.Label(
                    value='',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [118]:
global stored_choices, path

# stored_choices = {}
path = "../temp_storage.json"
back_path = "../backtracking.json"

In [119]:
with open(path, "r") as f:
    stored_choices = json.load(f)

In [120]:
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [121]:
# Read the CSV file into a DataFrame

df = pd.read_csv('new_format.csv', low_memory=False)

In [122]:
notebook_list = ["objective", "num_cams", "cam_strat_covar", "surv_dur_min_max", "sp_asymptote", 
                 "obj_targ_sp", "sp_info", "sp_occ_restr", "sp_hr_size", "sp_size", "sp_rarity", 
                 "sp_detprob_cat", "cam_independent"]

In [123]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list

def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [124]:
# gets the question based on value of other column

ques1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'question_text'].iloc[0]
opt1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'q_option_label'].iloc[0]
opt_codes1 = df.loc[df['question_code'] == 'surv_dur_mth_min', 'q_option_code'].iloc[0]

opt1 = split_and_trim(opt1)
opt_codes1 = split_and_trim(opt_codes1)

# ques2 = df.loc[df['question_code'] == 'num_cams_avail', 'question_text'].iloc[0]

In [125]:
# # Display question and options as widgets

# question_widget1 = widgets.HTML(value=f"<h3>{ques1}</h3>")
# options_widget1 = widgets.RadioButtons(options=opt1, value = None, layout={'width': 'max-content'})

# # question_widget2 = widgets.HTML(value=f"<h3>{ques2}</h3>")
# # options_widget2 = widgets.IntText(
# #                             value=0,  # Default value
# #                             description='Enter value:',
# #                             disabled=False,
# #                             layout={'width': 'max-content'}
# #                         )

In [126]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto')) #width='max-content'

In [127]:
# Factor widget

ques_button = create_expanded_button(ques1, 'success')
ques_button.style.button_color = 'lightblue'
ques_button.style.text_color = 'black'

option_1_button = create_expanded_button(opt1[0], 'success')
option_1_button.style.button_color = 'lightgreen'
option_1_button.style.text_color = 'black'

option_2_button = create_expanded_button(opt1[1], 'success')
option_2_button.style.button_color = 'red'
option_2_button.style.text_color = 'black'


factor_grid = GridspecLayout(4, 12, grid_gap='5px', height='200px')
factor_grid[1:3, 0:6] = ques_button
factor_grid[1, 7:9] = option_1_button
factor_grid[2, 7:9] = option_2_button

# factor_grid

In [128]:
output = widgets.Output()
def option_1_button_clicked(event):
    with output:
        clear_output()
        global options_widget2
        options_widget2 = widgets.IntText(
                            value=0,  # Default value
                            description='Enter value:',
                            disabled=False,
                            layout={'width': 'max-content'}
                        )
        
        display(options_widget2)
        
factor_grid[1, 7:9].on_click(option_1_button_clicked)

In [129]:
output = widgets.Output()
def option_2_button_clicked(event):
    with output:
        clear_output()
        selected_option = factor_grid[2, 7:9].description
        selected_option_index = opt1.index(selected_option)
        selected_option_code = opt_codes1[selected_option_index]
        
        stored_choices["surv_dur_mth_min"] = selected_option_code
        
factor_grid[2, 7:9].on_click(option_2_button_clicked)

In [130]:
# Create a button for going back to previous question

back_button = widgets.Button(description="Back")

In [131]:
# Create a buttons for submitting the question

submit_button = widgets.Button(description="Submit")

In [132]:
current_notebook_index = stored_choices["next_notebook_index"]

In [133]:
selected_objective = stored_choices['objective']

In [134]:
condition = df.loc[df['page_name'] == 'sp_asymptote', 'json_logic'].iloc[0]
condition_json = json.loads(condition)

In [135]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(link)

back_button.on_click(on_back_button_clicked)

In [136]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        global selected_option1
        
        link = "Submit your answer for next question"
        
        if selected_objective in condition_json['objective']:
            next_notebook_index = current_notebook_index + 1
        else:
            next_notebook_index = current_notebook_index + 2
        notebook_filename = notebook_list[next_notebook_index] + ".ipynb"
        
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Next</a>'''.format(notebook_filename))

        back_list.append("surv_dur_min_max")
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        
        if options_widget2.value > 0:
            stored_choices["surv_dur_mth_min"] = options_widget2.value
            options_widget2.value = 0
            
        stored_choices["next_notebook_index"] = next_notebook_index
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)

        display(link)

submit_button.on_click(on_button_clicked)

In [137]:
# Apply center alignment using CSS styling to the container

container = widgets.VBox(children=[factor_grid, output], layout=widgets.Layout(
    align_items='center'  # Center align vertically
))

button_container = widgets.HBox(children=[back_button, submit_button], layout=widgets.Layout(
    justify_content='center',  # Center align horizontally
))


display(vbox_headline)
display(container)
display(button_container)